Traduction du texte avec Translators et google.

In [1]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer

import RktnChallenge.preprocessing.translater
importlib.reload(RktnChallenge.preprocessing.translater)
from RktnChallenge.preprocessing.translater import LanguageDetector
from RktnChallenge.preprocessing.translater import Translator

import RktnChallenge.preprocessing.CopyWhereNan
importlib.reload(RktnChallenge.preprocessing.CopyWhereNan)
from RktnChallenge.preprocessing.CopyWhereNan import CopyWhereNan

import RktnChallenge.preprocessing.removeHTML
importlib.reload(RktnChallenge.preprocessing.removeHTML)
from RktnChallenge.preprocessing.removeHTML import removeHTML

2023-09-07 23:26:06.445208: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-07 23:26:06.477413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 23:26:07.060362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using region Normandy server backend.



On charge la base et init le support GPU / CPU

In [2]:
RktnModel = ModelTrainer("data.csv")

# On initialise le GPU disponible, la gestion mémoire
RktnModel.initGPU()

1 Physical GPUs, 1 Logical GPUs


2023-09-07 23:26:13.276442: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-07 23:26:13.408898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-07 23:26:13.409773: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-07 23:26:13.411895: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-07 23:26:13.412309: I tensorflow/compile

On détecte la langue de la source et range dans une nouvelle colonne

In [3]:
desc_removeHTML = removeHTML("description")
desi_removeHTML = removeHTML("designation")

desi_langueDetect = LanguageDetector("desi_langue","designation",confiance=0.8)
desc_langueDetect = LanguageDetector("desc_langue","description",confiance=0.8)


desi_Translator = Translator(dest = "tr_designation", src="designation",               
                             detected_lang="desi_langue", source = "google",verbose = False)

desc_Translator = Translator(dest = "tr_description", src="description",               
                             detected_lang="desc_langue", source = "google", verbose = False)



RktnModel.initPreprocess()
# unidecode et removeHTML
RktnModel.addPreprocessStep("desc removeHTML",desc_removeHTML)
RktnModel.addPreprocessStep("desi removeHTML",desi_removeHTML)
# decode la langue et traduit la designation
RktnModel.addPreprocessStep("detect langue designation",desi_langueDetect)
RktnModel.addPreprocessStep("translate designation",desi_Translator)
# decode la langue et traduit la description
RktnModel.addPreprocessStep("detect langue description",desc_langueDetect)
RktnModel.addPreprocessStep("translate description",desc_Translator)
RktnModel.preprocess()


print("langues detectees dans les designations: ",desi_langueDetect.langues)
print("langues detectees dans les descriptions: ",desc_langueDetect.langues)



/home/pec/Rktn/RktnChallenge/preprocessing/removeHTML.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  X[self.dest] = X[self.dest].apply(lambda x: "" if(pd.isna(x)) else unidecode(BeautifulSoup(x, "html.parser").text))
/home/pec/Rktn/RktnChallenge/preprocessing/removeHTML.py:17: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  X[self.dest] = X[self.dest].apply(lambda x: "" if(pd.isna(x)) else unidecode(BeautifulSoup(x, "html.parser").text))
/home/pec/Rktn/RktnChallenge/preprocessing/removeHTML.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  X[self.dest] = X[self.dest].apply(lambda x: "" if(pd.isna(x)) 

Loading lid Lang model


fin de detection
translating ... designation
fin de translation
fin de detection
translating ... description
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  https://placehold.it/100x70
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  ...
erreur:  NOTE : code d'activation a utiliser exclusivement sur un compte Steam valide connexion Internet requise.  A PROPOS DU JEU  Get the complete Earth 21x0 saga a milestone in gaming history including all expansion packs. Become a strategic commander at the highest level with World War II: Panzer Claws and World War III: Black Gold. And for a bit of variety head out into space with Planets under Attack.  EARTH 2160 - jeu disponible en francais anglais allemand italien espagnol   The future of mankind begins when the world ends...  After the destruction of the Earth in 2150 the survivors of the human race populate Mars - but this new homeland of theirs conceals

On vient de traduire dans tr_ ce qui n'est pas en francais, on complete avec le français:

In [12]:
# on copie les lignes qui n'ont pas été traduites (celles en français)
desc_copy = CopyWhereNan("tr_description","description")
desi_copy = CopyWhereNan("tr_designation","designation")

RktnModel.initPreprocess()
RktnModel.addPreprocessStep("desc copy french",desc_copy)
RktnModel.addPreprocessStep("desi copy french",desi_copy)
RktnModel.data = RktnModel.preprocess()


#on sauve
RktnModel.saveCSV("data_tr.csv")
